In [1]:
# import libraries
from src.preprocess.preprocess import *
from src.data_operations.rw_utils import read_from_csv, write_to_csv, read_from_excel
from src.data_operations.data_quality import create_data_quality_report
from src.data_operations.plots import plot_bar_chart, plot_wordcloud
import time
import numpy as np


In [2]:
# read data from source
data = read_from_csv("turkish_movie_sentiment_dataset.csv")
data.head()

Data is read. Len of the data 83227 and columns Index(['comment', 'film_name', 'point'], dtype='object')


,comment,film_name,point
0,\n Jean Reno denince zate...,Sevginin Gücü,"5,0"
1,\n Ekşın falan izlemek is...,Sevginin Gücü,"5,0"
2,\n Bu yapım hakkında öyle...,Sevginin Gücü,"5,0"
3,\n finali yeter... (sting...,Sevginin Gücü,"5,0"
4,\n Jean Reno..\r\nbu adam...,Sevginin Gücü,"5,0"


In [3]:
# get each label counts
df_label_counts = data["point"].value_counts()
df_label_counts

4,0    19958
5,0    15873
2,5    11433
3,5     8565
4,5     6983
3,0     5499
0,5     5150
1,0     4917
2,0     2433
1,5     1464
4,6      522
3,9      211
3,8      100
3,7       55
3,2       38
3,1       26
Name: point, dtype: int64

In [4]:
# visualize the sample counts of each label
labels = list(df_label_counts.to_frame().index)
counts = df_label_counts.values
plot_bar_chart(labels, counts, "Movie Reviews Point Distribution").show()

In [5]:
# create sentiment column from points
data["point"] = data["point"].astype(str).str.replace(",", ".")
data["point"] = data["point"].astype(float)
data["sentiment"] = np.where(data.point > 3.5, 1, 0)

In [6]:
# get each label counts
df_sent_counts = data["sentiment"].value_counts()
df_sent_counts

1    43702
0    39525
Name: sentiment, dtype: int64

In [7]:
# visualize the sample counts of each label
labels = list(df_sent_counts.to_frame().index)
counts = df_sent_counts.values
plot_bar_chart(labels, counts, "Movie Reviews Sentiment Distribution").show()

In [8]:
## Data Information
missing_value_df = create_data_quality_report(data, "sentiment")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83227 entries, 0 to 83226
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   comment    83227 non-null  object 
 1   film_name  83227 non-null  object 
 2   point      83227 non-null  float64
 3   sentiment  83227 non-null  int32  
dtypes: float64(1), int32(1), object(2)
memory usage: 2.2+ MB
#################################################
############ DATA QUALITY RESULT ################
#################################################

Number of sample in data set:83227.
Number of classes in data set: 2 and they are: [1 0].
Columns in data set:['comment', 'film_name', 'point', 'sentiment'].

None.



############## SUMMARY STATISTICS ###############

point     sentiment
count  83227.000000  83227.000000
mean       3.415253      0.525094
std        1.337989      0.499373
min        0.500000      0.000000
25%        2.500000      0.000000
50%        4.000000      1.000

In [9]:
missing_value_df

,column_name,percent_missing
comment,comment,0.0
film_name,film_name,0.0
point,point,0.0
sentiment,sentiment,0.0


In [10]:
# drop null values and duplicate values
data['comment'].replace('', np.nan, inplace=True)
data = data.dropna()
data = data.drop_duplicates()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83183 entries, 0 to 83226
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   comment    83183 non-null  object 
 1   film_name  83183 non-null  object 
 2   point      83183 non-null  float64
 3   sentiment  83183 non-null  int32  
dtypes: float64(1), int32(1), object(2)
memory usage: 2.9+ MB


In [11]:
# apply preprocess operations
preprocess_operations = [to_lower, remove_stopwords, handle_emojis, remove_hyperlink, remove_number, remove_punctuation, 
                         remove_whitespace,
                         replace_special_chars, remove_less_than_two]

In [12]:
# example result
data["comment"].values[12]

'\n                      Bir tarafta ailesini sevmeyen ve kötü alışkanlıkarı olan 12 yaşında bir kız, bir tarafta çok yalnız, tek aktivitesi adam öldürüp evde spor yapmak olan cahil olduğu kadar da zeki ve karizmatik olan bir suikastçi. \nİzledikten sonra yaklaşık 1 gün boyunca etkisinden çıkamadığım, her aklıma geldiğinde "ne filmdi be" diye iç geçirdiğim başyapıttır. Beğenmediğim pek bir tarafı yok.  Senaryonun orjinalinde Mathilda\'nın aşkına Leon da karşılık veriyor ve sevgili oluyorlar ama kızın ailesi bu senaryoyu reddettikleri için değiştiriliyor. Senaryo değişmeseydi nasıl olurdu merak etmiyor değilim. Sübyancı bir Leon bu kadar karizmatik olurmuydu acaba. Kız 12 değil 22 yaşında gibiydi. Oyunculuklar muhteşemdi. Leon\'un kızı eğitme sahneleride çok iyiydi. Kızın, ailesinin ölümü üzerine Leon\'un zilini çalıp lütfen aç diye sessizce ağlama sahnesinde heycanım zirve yapmıştı. Güzel bir filmdi izlemenizi tavsiye ederim.\n        \n            '

In [13]:
apply_preprocess_operations_to_corpus([data["comment"].values[12]], preprocess_operations)

['bir tarafta ailesini sevmeyen kötü alışkanlıkarı olan yaşında bir kız bir tarafta yalnız tek aktivitesi adam öldürüp evde spor yapmak olan cahil olduğu kadar zeki karizmatik olan bir suikastçi i̇zledikten sonra yaklaşık gün boyunca etkisinden çıkamadığım aklıma geldiğinde filmdi geçirdiğim başyapıttır beğenmediğim pek bir tarafı yok senaryonun orjinalinde mathildanın aşkına leon karşılık veriyor sevgili oluyorlar kızın ailesi senaryoyu reddettikleri değiştiriliyor senaryo değişmeseydi olurdu merak etmiyor değilim sübyancı bir leon kadar karizmatik olurmuydu acaba kız değil yaşında gibiydi oyunculuklar muhteşemdi leonun kızı eğitme sahneleride iyiydi kızın ailesinin ölümü üzerine leonun zilini çalıp lütfen sessizce ağlama sahnesinde heycanım zirve yapmıştı güzel bir filmdi izlemenizi tavsiye ederim']

In [14]:
# apply preprocess operation 
start = time.time()
for operation in preprocess_operations:
    data["comment"] = data["comment"].apply(operation)
print(f"Processed {len(data)} samples.\n")
print(f"It's took {(time.time()-start) / 60} minutes.")

Processed 83183 samples.

It's took 0.2490411917368571 minutes.


In [15]:
write_to_csv("preprocess_movie_sentiment.csv", data)

Data is wrote to path C:\Users\user\Desktop\YL\1.2\hesaplamalı_anabilim\ödev2\20501001\20501001\sentiment_analysis/data/, with name preprocess_movie_sentiment.csv


In [16]:
#plot_wordcloud(data.where(data.sentiment == 1).dropna()["comment"].values, title="Movie Data Sentiment Analysis Word Cloud Olumlu")

In [17]:
#plot_wordcloud(data.where(data.sentiment == 0).dropna()["comment"].values, title="Movie Data Sentiment Analysis Word Cloud Olumsuz")